<a href="https://colab.research.google.com/github/abubakry121/Audio_Generation/blob/master/StarGANv2_VC_train_audio_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

samuelsamsudinng_iemocap_emotion_speech_database_path = kagglehub.dataset_download('samuelsamsudinng/iemocap-emotion-speech-database')
bakrihassan_soundact_path = kagglehub.dataset_download('bakrihassan/soundact')
bakrihassan_iemocap_path = kagglehub.dataset_download('bakrihassan/iemocap')

print('Data source import complete.')


In [ ]:
!git clone https://github.com/koushik-elite/StarGANv2-VC-Custom.git StarGANv2-VC

Cloning into 'StarGANv2-VC'...
remote: Enumerating objects: 232, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 232 (delta 65), reused 59 (delta 59), pack-reused 153 (from 1)
Receiving objects: 100% (232/232), 108.19 MiB | 37.57 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [ ]:
!pwd

/kaggle/working


In [ ]:
%cd StarGANv2-VC

/kaggle/working/StarGANv2-VC


In [ ]:
!pip install SoundFile torchaudio munch parallel_wavegan torch pydub pyyaml click librosa

In [ ]:
import os
import pandas as pd

def dataset_dict(dir_path, label, result):
    for file_path in os.listdir(dir_path):
        # check if current file_path is a file
        full_path = os.path.join(dir_path, file_path)
        if os.path.isfile(full_path):
            # add filename to list
            result.append([full_path, label])
    return result

# directory/folder path

# list to store files
res = []

# res = dataset_dict('/kaggle/input/starganv2-vc-my/Data/Data/p225/', 0, res)
# res = dataset_dict('/kaggle/input/starganv2-vc-my/Data/Data/p226/', 1, res)
# res = dataset_dict('/kaggle/input/starganv2-vc-my/Data/Data/p227/', 2, res)

res = dataset_dict('/kaggle/input/soundact', 0, res)
#res = dataset_dict('/kaggle/input/starganv2-vc-my/Tamil/suchi/', 1, res)
#res = dataset_dict('/kaggle/input/starganv2-vc-my/Tamil/barathi/', 2, res)
#res = dataset_dict('/kaggle/input/starganv2-vc-my/Tamil/kapila/', 3, res)
#res = dataset_dict('/kaggle/input/starganv2-vc-my/Tamil/nith/', 4, res)
#res = dataset_dict('/kaggle/input/starganv2-vc-my/Tamil/kick/', 5, res)
# Iterate directory

data_df = pd.DataFrame(res)
data_df.head()

""


In [ ]:
data_df = data_df.sample(frac = 1).reset_index(drop=True)
data_df.head()

""


In [ ]:
size = data_df.shape[0]
split_no = int(size * 0.95)

In [ ]:
train_df = data_df[:split_no]
val_df = data_df[split_no:]

In [ ]:
train_df.to_csv('/kaggle/working/StarGANv2-VC/Data/train_list.txt', sep='|', index=False, header=None)
val_df.to_csv('/kaggle/working/StarGANv2-VC/Data/val_list.txt', sep='|', index=False, header=None)

In [ ]:
import yaml

fname = "./Configs/config.yml"

# stream = open(fname, 'r')
# data = yaml.load(stream)

with open(fname, "r") as stream:
    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

data['F0_path'] = '/kaggle/input/pitchextractor-output/PitchExtractor/Checkpoint/epoch_00100.pth'
data['ASR_config'] = '/kaggle/input/auxiliaryasr-tamil-ouput/AuxiliaryASR-Customized/Checkpoint/config.yml'
data['ASR_path'] = '/kaggle/input/auxiliaryasr-tamil-ouput/AuxiliaryASR-Customized/Checkpoint/epoch_00010.pth'

with open(fname, 'w') as yaml_file:
    yaml_file.write( yaml.dump(data, default_flow_style=False))

In [ ]:
# import yaml

# data = dict(
#     log_dir = 'Models/VCTK20',
#     save_freq = 20,
#     device = 'cuda',
#     epochs = 100,
#     batch_size = 10,
#     pretrained_model = '',
#     load_only_params = False,
#     fp16_run = True,
#     train_data = "Data/train_list.txt",
#     val_data = "Data/val_list.txt",
#     F0_path = "Utils/JDC/bst.t7",
#     ASR_config = "Utils/ASR/config.yml",
#     ASR_path = "Utils/ASR/epoch_00100.pth",
#     preprocess_params = dict(
#         sr = 24000,
#         spect_params = dict(
#             n_fft = 2048,
#             win_length = 1200,
#             hop_length = 300,
#         )
#     ),
#     model_params = dict(
#         dim_in = 64,
#         style_dim = 64,
#         latent_dim = 16,
#         num_domains = 20,
#         max_conv_dim = 512,
#         n_repeat = 4,
#         w_hpf = 0,
#         F0_channel = 256,
#     ),
#     loss_params = dict(
#         g_loss = dict(
#             lambda_sty = 1,
#             lambda_cyc = 5,
#             lambda_ds = 1,
#             lambda_norm = 1,
#             lambda_asr = 10,
#             lambda_f0 = 5,
#             lambda_f0_sty = 0.1,
#             lambda_adv = 2,
#             lambda_adv_cls = 0.5,
#             norm_bias = 0.5,
#         ),
#         d_loss = dict(
#             lambda_reg = 1,
#             lambda_adv_cls = 0.1,
#             lambda_con_reg = 10,
#         ),
#         adv_cls_epoch = 50,
#         con_reg_epoch = 30,
#     ),
#     optimizer_params = dict(
#         lr = 0.0001
#     )
# )

# with open('./Configs/config.yml', 'w') as outfile:
#     yaml.dump(data, outfile, default_flow_style=False)

In [ ]:
# import os
# import os.path as osp
# import re
# import sys
# import yaml
# import shutil
# import numpy as np
# import torch
# import click
# import warnings
# warnings.simplefilter('ignore')

# from functools import reduce
# from munch import Munch

# from meldataset import build_dataloader
# from optimizers import build_optimizer
# from models import build_model
# from trainer import Trainer
# from torch.utils.tensorboard import SummaryWriter

# from Utils.ASR.models import ASRCNN
# from Utils.JDC.model import JDCNet

# import logging
# from logging import StreamHandler
# logger = logging.getLogger(__name__)
# logger.setLevel(logging.DEBUG)
# handler = StreamHandler()
# handler.setLevel(logging.DEBUG)
# logger.addHandler(handler)

# torch.backends.cudnn.benchmark = True #

In [ ]:
# log_dir = data['log_dir']
# if not osp.exists(log_dir): os.makedirs(log_dir, exist_ok=True)
# # shutil.copy(config_path, osp.join(log_dir, osp.basename(config_path)))
# writer = SummaryWriter(log_dir + "/tensorboard")

# # write logs
# file_handler = logging.FileHandler(osp.join(log_dir, 'train.log'))
# file_handler.setLevel(logging.DEBUG)
# file_handler.setFormatter(logging.Formatter('%(levelname)s:%(asctime)s: %(message)s'))
# logger.addHandler(file_handler)

In [ ]:
# batch_size = data["batch_size"]
# device = data["device"]
# epochs = data["epochs"]
# save_freq = data["save_freq"]
# train_path = '/kaggle/working/StarGANv2-VC/Data/train_list.txt'
# val_path = '/kaggle/working/StarGANv2-VC/Data/train_list.txt'
# stage = 'star'
# fp16_run = data["fp16_run"]

In [ ]:
# train_list, val_list = get_data_path_list(train_path, val_path)
# train_list

In [ ]:
# train_dataloader = build_dataloader(train_list, batch_size=batch_size, num_workers=0, device=device)
# val_dataloader = build_dataloader(val_list, batch_size=batch_size, validation=True, num_workers=0, device=device)

In [ ]:
# # load pretrained ASR model
# ASR_config = "/kaggle/input/auxiliaryasr-tamil-ouput/AuxiliaryASR-Customized/Configs/config.yml"
# ASR_path = "/kaggle/input/auxiliaryasr-tamil-ouput/AuxiliaryASR-Customized/Checkpoint/epoch_00100.pth"
# with open(ASR_config) as f:
#     ASR_config = yaml.safe_load(f)

# ASR_model_config = ASR_config['model_params']
# ASR_model = ASRCNN(**ASR_model_config)
# params = torch.load(ASR_path, map_location='cpu')['model']
# ASR_model.load_state_dict(params)
# _ = ASR_model.eval()

# # load pretrained F0 model
# # F0_path = "/kaggle/input/pitchextractor-output/PitchExtractor/Checkpoint/epoch_00100.pth"
# # F0_path = "/kaggle/working/StarGANv2-VC/Utils/JDC/bst.t7"
# F0_path = "/kaggle/input/pitchextractor-output/PitchExtractor/Checkpoint/epoch_00100.pth"
# F0_model = JDCNet(num_class=1, seq_len=192)

# params = torch.load(F0_path, map_location='cpu')['model']
# F0_model.load_state_dict(params)

In [ ]:
# # build model
# model, model_ema = build_model(Munch(data["model_params"]), F0_model, ASR_model)
# scheduler_params = {
#     "max_lr": float(data['optimizer_params'].get('lr', 2e-4)),
#     "pct_start": float(data['optimizer_params'].get('pct_start', 0.0)),
#     "epochs": epochs,
#     "steps_per_epoch": len(train_dataloader),
# }
# _ = [model[key].to(device) for key in model]
# _ = [model_ema[key].to(device) for key in model_ema]
# scheduler_params_dict = {key: scheduler_params.copy() for key in model}
# scheduler_params_dict['mapping_network']['max_lr'] = 2e-6
# optimizer = build_optimizer({key: model[key].parameters() for key in model}, scheduler_params_dict=scheduler_params_dict)

In [ ]:
# trainer = Trainer(args=Munch(data['loss_params']), model=model,
#                             model_ema=model_ema,
#                             optimizer=optimizer,
#                             device=device,
#                             train_dataloader=train_dataloader,
#                             val_dataloader=val_dataloader,
#                             logger=logger,
#                             fp16_run=fp16_run)

In [ ]:
# for step, (x, y) in enumerate(train_dataloader):
#     print(x)

In [ ]:
# for _ in range(1, epochs+1):
#     epoch = trainer.epochs
#     train_results = trainer._train_epoch()
#     eval_results = trainer._eval_epoch()
#     results = train_results.copy()
#     results.update(eval_results)
#     logger.info('--- epoch %d ---' % epoch)
#     for key, value in results.items():
#         if isinstance(value, float):
#             logger.info('%-15s: %.4f' % (key, value))
#             writer.add_scalar(key, value, epoch)
#         else:
#             for v in value:
#                 writer.add_figure('eval_spec', v, epoch)
#     if (epoch % save_freq) == 0:
#         trainer.save_checkpoint(osp.join(log_dir, 'epoch_%05d.pth' % epoch))

In [ ]:
# trainer.load_checkpoint(data['pretrained_model'], load_only_params=True)

In [ ]:
!python train.py --config_path ./Configs/config.yml

Traceback (most recent call last):
  File "/kaggle/working/StarGANv2-VC/train.py", line 156, in <module>
    main()
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/kaggle/working/StarGANv2-VC/train.py", line 66, in main
    train_dataloader = build_dataloader(train_list,
  File "/kaggle/working/StarGANv2-VC/meldataset.py", line 148, in build_dataloader
    data_loader = DataLoader(dataset,
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 351, in __init__
    sampler = RandomSampler(dataset, generator=gen

In [ ]:
!python preprocess_audio.py --iemocap_dir /kaggle/input/soundact

In [ ]:
# preprocess_audio_kaggle.py
import os
import librosa
import soundfile as sf

# ===== CONFIG =====
# Folder where you uploaded your audio dataset on Kaggle
INPUT_PATH = '/kaggle/input/soundact'   # Replace with your dataset folder name
OUTPUT_PATH = '/kaggle/working/preprocessed/'  # Folder to save preprocessed audio
SR = 16000  # Sampling rate for audio
DURATION = 4.0  # seconds, to standardize length

# ===== CREATE OUTPUT FOLDER =====
os.makedirs(OUTPUT_PATH, exist_ok=True)

# ===== CHECK INPUT =====
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"Input folder not found: {INPUT_PATH}")

files = [f for f in os.listdir(INPUT_PATH) if f.endswith('.wav')]
if len(files) == 0:
    raise ValueError(f"No .wav files found in {INPUT_PATH}")

print(f"Found {len(files)} audio files. Starting preprocessing...")

# ===== PREPROCESSING =====
for i, file_name in enumerate(files):
    file_path = os.path.join(INPUT_PATH, file_name)

    # Load audio
    y, sr = librosa.load(file_path, sr=SR)

    # Standardize length
    max_len = int(SR * DURATION)
    if len(y) > max_len:
        y = y[:max_len]
    else:
        y = librosa.util.fix_length(y, max_len)

    # Save preprocessed file
    out_file = os.path.join(OUTPUT_PATH, file_name)
    sf.write(out_file, y, SR)

    if (i+1) % 10 == 0 or (i+1) == len(files):
        print(f"Processed {i+1}/{len(files)} files")

print(f"All files saved to {OUTPUT_PATH}")
